In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

In [2]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def prepr_image(img_path):
    img = Image.open(img_path)
    return normalize(to_tensor(scaler(img)))

model_ft = models.vgg16(pretrained=True)
feature_extractor = nn.Sequential(*list(model_ft.children())[:-1], nn.Flatten(), nn.Linear(512 * 49, 2048), nn.ReLU(), nn.Linear(2048, 2048), nn.ReLU(), nn.Linear(2048, 80)).cuda()

In [3]:
def read_images():
    train_dataset = torch.zeros([48000, 3, 224, 224], dtype=torch.float)
    for j in range(80):
        folder = 'train/' + str(j + 1) + '/'
        for i in range(600):
            train_dataset[j * 600 + i] = prepr_image(folder + str(i) + '.jpg')
        print('Training {} class preprocessed'.format(j))
    return train_dataset

train_dataset = read_images()
order = np.array([], dtype=int)
def batch_loader(batch_num):
    indices = np.array(order[batch_num * batch_size:(batch_num + 1) * batch_size], dtype=int)
    batch_images = train_dataset[indices].cuda()
    batch_labels = torch.tensor([indices[i] // 600 for i in range(batch_size)], dtype=int).cuda()
    return (batch_images, batch_labels)

Training 0 class preprocessed
Training 1 class preprocessed
Training 2 class preprocessed
Training 3 class preprocessed
Training 4 class preprocessed
Training 5 class preprocessed
Training 6 class preprocessed
Training 7 class preprocessed
Training 8 class preprocessed
Training 9 class preprocessed
Training 10 class preprocessed
Training 11 class preprocessed
Training 12 class preprocessed
Training 13 class preprocessed
Training 14 class preprocessed
Training 15 class preprocessed
Training 16 class preprocessed
Training 17 class preprocessed
Training 18 class preprocessed
Training 19 class preprocessed
Training 20 class preprocessed
Training 21 class preprocessed
Training 22 class preprocessed
Training 23 class preprocessed
Training 24 class preprocessed
Training 25 class preprocessed
Training 26 class preprocessed
Training 27 class preprocessed
Training 28 class preprocessed
Training 29 class preprocessed
Training 30 class preprocessed
Training 31 class preprocessed
Training 32 class 

In [ ]:
rate = 0.001
optimizer = torch.optim.SGD(feature_extractor.parameters(), lr=rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

acc_list = []

num_epochs = 30
batch_size = 100
for epoch in range(num_epochs):
    torch.save(feature_extractor.state_dict(), 'vgg16_extractor_tmp')
    order = np.array([i for i in range(48000)], dtype=int) # 5
    np.random.shuffle(order)
    if (epoch == 15):
        rate *= 0.1
        optimizer = torch.optim.SGD(feature_extractor.parameters(), lr=rate, momentum=0.9)
        torch.save(feature_extractor.state_dict(), 'vgg16_extractor_15')
    for i in range(48000 // batch_size): # 6
        batch = batch_loader(i)
        outs = feature_extractor(batch[0])
        loss = criterion(outs, batch[1])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print('Epoch {}, batch {}, Loss = {}'.format(epoch, i, loss))
        
torch.save(feature_extractor.state_dict(), 'vgg16_extractor_30')

Epoch 0, batch 0, Loss = 4.409200191497803
Epoch 0, batch 1, Loss = 4.389795780181885
Epoch 0, batch 2, Loss = 4.3941826820373535
Epoch 0, batch 3, Loss = 4.37318754196167
Epoch 0, batch 4, Loss = 4.379986763000488
Epoch 0, batch 5, Loss = 4.379972457885742
Epoch 0, batch 6, Loss = 4.380722999572754
Epoch 0, batch 7, Loss = 4.385516166687012
Epoch 0, batch 8, Loss = 4.378541946411133
Epoch 0, batch 9, Loss = 4.387274265289307
Epoch 0, batch 10, Loss = 4.371376037597656
Epoch 0, batch 11, Loss = 4.355104446411133
Epoch 0, batch 12, Loss = 4.375634670257568
Epoch 0, batch 13, Loss = 4.371252059936523
Epoch 0, batch 14, Loss = 4.383676528930664
Epoch 0, batch 15, Loss = 4.359407424926758
Epoch 0, batch 16, Loss = 4.369813442230225
Epoch 0, batch 17, Loss = 4.363927841186523
Epoch 0, batch 18, Loss = 4.372076511383057
Epoch 0, batch 19, Loss = 4.357416152954102
Epoch 0, batch 20, Loss = 4.377791881561279
Epoch 0, batch 21, Loss = 4.373626232147217
Epoch 0, batch 22, Loss = 4.35240554809570

Epoch 0, batch 183, Loss = 1.2649016380310059
Epoch 0, batch 184, Loss = 1.1526803970336914
Epoch 0, batch 185, Loss = 1.4679512977600098
Epoch 0, batch 186, Loss = 1.0782930850982666
Epoch 0, batch 187, Loss = 1.2219990491867065
Epoch 0, batch 188, Loss = 1.1804916858673096
Epoch 0, batch 189, Loss = 0.9459519982337952
Epoch 0, batch 190, Loss = 0.9222424030303955
Epoch 0, batch 191, Loss = 1.2281640768051147
Epoch 0, batch 192, Loss = 1.3303755521774292
Epoch 0, batch 193, Loss = 1.0981495380401611
Epoch 0, batch 194, Loss = 1.1922794580459595
Epoch 0, batch 195, Loss = 1.040817141532898
Epoch 0, batch 196, Loss = 1.2130588293075562
Epoch 0, batch 197, Loss = 1.3039239645004272
Epoch 0, batch 198, Loss = 1.4205459356307983
Epoch 0, batch 199, Loss = 1.4385911226272583
Epoch 0, batch 200, Loss = 1.1904940605163574
Epoch 0, batch 201, Loss = 1.4337437152862549
Epoch 0, batch 202, Loss = 0.9497926235198975
Epoch 0, batch 203, Loss = 1.1274722814559937
Epoch 0, batch 204, Loss = 0.896727

Epoch 0, batch 362, Loss = 0.7426280975341797
Epoch 0, batch 363, Loss = 0.7824631333351135
Epoch 0, batch 364, Loss = 1.1435558795928955
Epoch 0, batch 365, Loss = 1.2192881107330322
Epoch 0, batch 366, Loss = 0.981180727481842
Epoch 0, batch 367, Loss = 0.8131458163261414
Epoch 0, batch 368, Loss = 0.9407036304473877
Epoch 0, batch 369, Loss = 0.8907370567321777
Epoch 0, batch 370, Loss = 1.063222050666809
Epoch 0, batch 371, Loss = 0.9909271001815796
Epoch 0, batch 372, Loss = 0.8338554501533508
Epoch 0, batch 373, Loss = 0.9240420460700989
Epoch 0, batch 374, Loss = 1.1116249561309814
Epoch 0, batch 375, Loss = 1.173566222190857
Epoch 0, batch 376, Loss = 0.9816494584083557
Epoch 0, batch 377, Loss = 0.9224510192871094
Epoch 0, batch 378, Loss = 0.9391890168190002
Epoch 0, batch 379, Loss = 0.9899947643280029
Epoch 0, batch 380, Loss = 1.1311880350112915
Epoch 0, batch 381, Loss = 1.2651523351669312
Epoch 0, batch 382, Loss = 0.8263686895370483
Epoch 0, batch 383, Loss = 1.15856981

Epoch 1, batch 62, Loss = 0.5072488784790039
Epoch 1, batch 63, Loss = 0.5328404903411865
Epoch 1, batch 64, Loss = 0.3161299228668213
Epoch 1, batch 65, Loss = 0.529586911201477
Epoch 1, batch 66, Loss = 0.4068772494792938
Epoch 1, batch 67, Loss = 0.6462851166725159
Epoch 1, batch 68, Loss = 0.5532292723655701
Epoch 1, batch 69, Loss = 0.6461130380630493
Epoch 1, batch 70, Loss = 0.32404908537864685
Epoch 1, batch 71, Loss = 0.4159785509109497
Epoch 1, batch 72, Loss = 0.627215564250946
Epoch 1, batch 73, Loss = 0.44016334414482117
Epoch 1, batch 74, Loss = 0.42925721406936646
Epoch 1, batch 75, Loss = 0.3411622643470764
Epoch 1, batch 76, Loss = 0.5066144466400146
Epoch 1, batch 77, Loss = 0.4083431661128998
Epoch 1, batch 78, Loss = 0.6353079080581665
Epoch 1, batch 79, Loss = 0.593636691570282
Epoch 1, batch 80, Loss = 0.43670105934143066
Epoch 1, batch 81, Loss = 0.5729449987411499
Epoch 1, batch 82, Loss = 0.3873101770877838
Epoch 1, batch 83, Loss = 0.3762670159339905
Epoch 1, 

Epoch 1, batch 241, Loss = 0.5948642492294312
Epoch 1, batch 242, Loss = 0.691935122013092
Epoch 1, batch 243, Loss = 0.48554831743240356
Epoch 1, batch 244, Loss = 0.6267971992492676
Epoch 1, batch 245, Loss = 0.472951740026474
Epoch 1, batch 246, Loss = 0.5864196419715881
Epoch 1, batch 247, Loss = 0.4802652597427368
Epoch 1, batch 248, Loss = 0.5115737915039062
Epoch 1, batch 249, Loss = 0.5433542132377625
Epoch 1, batch 250, Loss = 0.4336911737918854
Epoch 1, batch 251, Loss = 0.46792206168174744
Epoch 1, batch 252, Loss = 0.6100144386291504
Epoch 1, batch 253, Loss = 0.3496074378490448
Epoch 1, batch 254, Loss = 0.2289835810661316
Epoch 1, batch 255, Loss = 0.2586279809474945
Epoch 1, batch 256, Loss = 0.4703855812549591
Epoch 1, batch 257, Loss = 0.3541695773601532
Epoch 1, batch 258, Loss = 0.37083935737609863
Epoch 1, batch 259, Loss = 0.37673377990722656
Epoch 1, batch 260, Loss = 0.4653271436691284
Epoch 1, batch 261, Loss = 0.4503495693206787
Epoch 1, batch 262, Loss = 0.365

Epoch 1, batch 419, Loss = 0.6564691066741943
Epoch 1, batch 420, Loss = 0.3944406807422638
Epoch 1, batch 421, Loss = 0.6318871378898621
Epoch 1, batch 422, Loss = 0.535788357257843
Epoch 1, batch 423, Loss = 0.3692704737186432
Epoch 1, batch 424, Loss = 0.3205541968345642
Epoch 1, batch 425, Loss = 0.3678886890411377
Epoch 1, batch 426, Loss = 0.32316476106643677
Epoch 1, batch 427, Loss = 0.30637526512145996
Epoch 1, batch 428, Loss = 0.23311281204223633
Epoch 1, batch 429, Loss = 0.5327327251434326
Epoch 1, batch 430, Loss = 0.6194719076156616
Epoch 1, batch 431, Loss = 0.6172975301742554
Epoch 1, batch 432, Loss = 0.3763623535633087
Epoch 1, batch 433, Loss = 0.38956984877586365
Epoch 1, batch 434, Loss = 0.3812969923019409
Epoch 1, batch 435, Loss = 0.48690056800842285
Epoch 1, batch 436, Loss = 0.4681032598018646
Epoch 1, batch 437, Loss = 0.34459248185157776
Epoch 1, batch 438, Loss = 0.5967385172843933
Epoch 1, batch 439, Loss = 0.4315342307090759
Epoch 1, batch 440, Loss = 0.

Epoch 2, batch 117, Loss = 0.09374648332595825
Epoch 2, batch 118, Loss = 0.049230899661779404
Epoch 2, batch 119, Loss = 0.20625348389148712
Epoch 2, batch 120, Loss = 0.058922577649354935
Epoch 2, batch 121, Loss = 0.08614499121904373
Epoch 2, batch 122, Loss = 0.09761309623718262
Epoch 2, batch 123, Loss = 0.04370807111263275
Epoch 2, batch 124, Loss = 0.0892871841788292
Epoch 2, batch 125, Loss = 0.06900728493928909
Epoch 2, batch 126, Loss = 0.10619097948074341
Epoch 2, batch 127, Loss = 0.08025360107421875
Epoch 2, batch 128, Loss = 0.0635400116443634
Epoch 2, batch 129, Loss = 0.10169325768947601
Epoch 2, batch 130, Loss = 0.09954509884119034
Epoch 2, batch 131, Loss = 0.08364421129226685
Epoch 2, batch 132, Loss = 0.05982285365462303
Epoch 2, batch 133, Loss = 0.054985638707876205
Epoch 2, batch 134, Loss = 0.07886216789484024
Epoch 2, batch 135, Loss = 0.09589803963899612
Epoch 2, batch 136, Loss = 0.04589204862713814
Epoch 2, batch 137, Loss = 0.15938535332679749
Epoch 2, bat

Epoch 2, batch 292, Loss = 0.09311485290527344
Epoch 2, batch 293, Loss = 0.051778629422187805
Epoch 2, batch 294, Loss = 0.050056539475917816
Epoch 2, batch 295, Loss = 0.06499550491571426
Epoch 2, batch 296, Loss = 0.1288238763809204
Epoch 2, batch 297, Loss = 0.07985875010490417
Epoch 2, batch 298, Loss = 0.16268496215343475
Epoch 2, batch 299, Loss = 0.1799045354127884
Epoch 2, batch 300, Loss = 0.08205705881118774
Epoch 2, batch 301, Loss = 0.06919331848621368
Epoch 2, batch 302, Loss = 0.11080119758844376
Epoch 2, batch 303, Loss = 0.08538470417261124
Epoch 2, batch 304, Loss = 0.05839606374502182
Epoch 2, batch 305, Loss = 0.06514450162649155
Epoch 2, batch 306, Loss = 0.0689389556646347
Epoch 2, batch 307, Loss = 0.16408812999725342
Epoch 2, batch 308, Loss = 0.10002259910106659
Epoch 2, batch 309, Loss = 0.059347931295633316
Epoch 2, batch 310, Loss = 0.06237048655748367
Epoch 2, batch 311, Loss = 0.09638507664203644
Epoch 2, batch 312, Loss = 0.07985714077949524
Epoch 2, batc

Epoch 2, batch 467, Loss = 0.07849995791912079
Epoch 2, batch 468, Loss = 0.06723003089427948
Epoch 2, batch 469, Loss = 0.0630192905664444
Epoch 2, batch 470, Loss = 0.05110800638794899
Epoch 2, batch 471, Loss = 0.09755208343267441
Epoch 2, batch 472, Loss = 0.08033084869384766
Epoch 2, batch 473, Loss = 0.09346089512109756
Epoch 2, batch 474, Loss = 0.09953819960355759
Epoch 2, batch 475, Loss = 0.08050104230642319
Epoch 2, batch 476, Loss = 0.0976901426911354
Epoch 2, batch 477, Loss = 0.078672856092453
Epoch 2, batch 478, Loss = 0.16076737642288208
Epoch 2, batch 479, Loss = 0.18745297193527222
Epoch 3, batch 0, Loss = 0.012145919725298882
Epoch 3, batch 1, Loss = 0.04113413020968437
Epoch 3, batch 2, Loss = 0.02075907774269581
Epoch 3, batch 3, Loss = 0.1701582968235016
Epoch 3, batch 4, Loss = 0.017460232600569725
Epoch 3, batch 5, Loss = 0.02346339263021946
Epoch 3, batch 6, Loss = 0.04877609387040138
Epoch 3, batch 7, Loss = 0.056340016424655914
Epoch 3, batch 8, Loss = 0.0251

Epoch 3, batch 162, Loss = 0.03306379169225693
Epoch 3, batch 163, Loss = 0.009357137605547905
Epoch 3, batch 164, Loss = 0.01810585893690586
Epoch 3, batch 165, Loss = 0.07624683529138565
Epoch 3, batch 166, Loss = 0.026692314073443413
Epoch 3, batch 167, Loss = 0.00793188065290451
Epoch 3, batch 168, Loss = 0.019050616770982742
Epoch 3, batch 169, Loss = 0.034334659576416016
Epoch 3, batch 170, Loss = 0.007051429711282253
Epoch 3, batch 171, Loss = 0.033135510981082916
Epoch 3, batch 172, Loss = 0.03832891583442688
Epoch 3, batch 173, Loss = 0.03067178651690483
Epoch 3, batch 174, Loss = 0.008487091399729252
Epoch 3, batch 175, Loss = 0.06074605882167816
Epoch 3, batch 176, Loss = 0.006187743972986937
Epoch 3, batch 177, Loss = 0.044146984815597534
Epoch 3, batch 178, Loss = 0.016912316903471947
Epoch 3, batch 179, Loss = 0.004596862941980362
Epoch 3, batch 180, Loss = 0.018290556967258453
Epoch 3, batch 181, Loss = 0.04566716030240059
Epoch 3, batch 182, Loss = 0.023581000044941902


In [4]:
feature_extractor.load_state_dict(torch.load('vgg16_extractor_tmp'))

<All keys matched successfully>

In [ ]:
rate = 0.001
optimizer = torch.optim.SGD(feature_extractor.parameters(), lr=rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

acc_list = []

num_epochs = 18
batch_size = 100
for epoch in range(num_epochs):
    torch.save(feature_extractor.state_dict(), 'vgg16_extractor_tmp')
    order = np.array([i for i in range(48000)], dtype=int) # 5
    np.random.shuffle(order)
    if (epoch == 8):
        rate *= 0.1
        optimizer = torch.optim.SGD(feature_extractor.parameters(), lr=rate, momentum=0.9)
        torch.save(feature_extractor.state_dict(), 'vgg16_extractor_10')
    for i in range(48000 // batch_size): # 6
        batch = batch_loader(i)
        outs = feature_extractor(batch[0])
        loss = criterion(outs, batch[1])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print('Epoch {}, batch {}, Loss = {}'.format(epoch, i, loss))
        
torch.save(feature_extractor.state_dict(), 'vgg16_extractor_20')

Epoch 0, batch 0, Loss = 0.004683151375502348
Epoch 0, batch 1, Loss = 0.0012027740012854338
Epoch 0, batch 2, Loss = 0.0022182082757353783
Epoch 0, batch 3, Loss = 0.01958460733294487
Epoch 0, batch 4, Loss = 0.0009140014881268144
Epoch 0, batch 5, Loss = 0.019175034016370773
Epoch 0, batch 6, Loss = 0.00493451114743948
Epoch 0, batch 7, Loss = 0.01288946159183979
Epoch 0, batch 8, Loss = 0.002407245570793748
Epoch 0, batch 9, Loss = 0.0062941075302660465
Epoch 0, batch 10, Loss = 0.0016515064053237438
Epoch 0, batch 11, Loss = 0.0018493462121114135
Epoch 0, batch 12, Loss = 0.002555189188569784
Epoch 0, batch 13, Loss = 0.0067373658530414104
Epoch 0, batch 14, Loss = 0.0049936771392822266
Epoch 0, batch 15, Loss = 0.0241794865578413
Epoch 0, batch 16, Loss = 0.0021889114286750555
Epoch 0, batch 17, Loss = 0.0015193080762401223
Epoch 0, batch 18, Loss = 0.002249660436064005
Epoch 0, batch 19, Loss = 0.05380247160792351
Epoch 0, batch 20, Loss = 0.0020657635759562254
Epoch 0, batch 21,

Epoch 0, batch 171, Loss = 0.0003238582576159388
Epoch 0, batch 172, Loss = 0.000978498486801982
Epoch 0, batch 173, Loss = 0.05714485049247742
Epoch 0, batch 174, Loss = 0.0006210899446159601
Epoch 0, batch 175, Loss = 0.0019070911221206188
Epoch 0, batch 176, Loss = 0.00030820845859125257
Epoch 0, batch 177, Loss = 0.0013041878119111061
Epoch 0, batch 178, Loss = 0.0002749442937783897
Epoch 0, batch 179, Loss = 0.0025623703841120005
Epoch 0, batch 180, Loss = 0.0024859237018972635
Epoch 0, batch 181, Loss = 0.03231944888830185
Epoch 0, batch 182, Loss = 0.0007581329555250704
Epoch 0, batch 183, Loss = 0.0012140751350671053
Epoch 0, batch 184, Loss = 0.16566841304302216
Epoch 0, batch 185, Loss = 0.0012081909226253629
Epoch 0, batch 186, Loss = 0.0008967590401880443
Epoch 0, batch 187, Loss = 0.0013739109272137284
Epoch 0, batch 188, Loss = 0.0017715358408167958
Epoch 0, batch 189, Loss = 0.009233512915670872
Epoch 0, batch 190, Loss = 0.0009188842959702015
Epoch 0, batch 191, Loss = 

Epoch 0, batch 341, Loss = 0.001174497650936246
Epoch 0, batch 342, Loss = 0.01807117462158203
Epoch 0, batch 343, Loss = 0.0015499591827392578
Epoch 0, batch 344, Loss = 0.002121067140251398
Epoch 0, batch 345, Loss = 0.00028545380337163806
Epoch 0, batch 346, Loss = 0.004090270958840847
Epoch 0, batch 347, Loss = 0.0033689308911561966
Epoch 0, batch 348, Loss = 0.0025665760040283203
Epoch 0, batch 349, Loss = 0.0013181590475142002
Epoch 0, batch 350, Loss = 0.0005544853047467768
Epoch 0, batch 351, Loss = 0.0011106872698292136
Epoch 0, batch 352, Loss = 0.008490047417581081
Epoch 0, batch 353, Loss = 0.0005114555242471397
Epoch 0, batch 354, Loss = 0.0021584797650575638
Epoch 0, batch 355, Loss = 0.0006837368127889931
Epoch 0, batch 356, Loss = 0.001024160417728126
Epoch 0, batch 357, Loss = 0.021119004115462303
Epoch 0, batch 358, Loss = 0.0026606940664350986
Epoch 0, batch 359, Loss = 0.0017427349230274558
Epoch 0, batch 360, Loss = 0.0008274555439129472
Epoch 0, batch 361, Loss = 

Epoch 1, batch 31, Loss = 0.0012277317000553012
Epoch 1, batch 32, Loss = 0.0025559901259839535
Epoch 1, batch 33, Loss = 0.002550964243710041
Epoch 1, batch 34, Loss = 0.0016227626474574208
Epoch 1, batch 35, Loss = 0.0012765408027917147
Epoch 1, batch 36, Loss = 0.0010016822488978505
Epoch 1, batch 37, Loss = 0.003998012747615576
Epoch 1, batch 38, Loss = 0.0013000392355024815
Epoch 1, batch 39, Loss = 0.0013260364066809416
Epoch 1, batch 40, Loss = 0.0008425331325270236
Epoch 1, batch 41, Loss = 0.001862602191977203
Epoch 1, batch 42, Loss = 0.0016450691036880016
Epoch 1, batch 43, Loss = 0.0005269718240015209
Epoch 1, batch 44, Loss = 0.0016437912127003074
Epoch 1, batch 45, Loss = 0.0005857181386090815
Epoch 1, batch 46, Loss = 0.0031833553221076727
Epoch 1, batch 47, Loss = 0.0005975532694719732
Epoch 1, batch 48, Loss = 0.000419025425799191
Epoch 1, batch 49, Loss = 0.0015725516714155674
Epoch 1, batch 50, Loss = 0.0009750175522640347
Epoch 1, batch 51, Loss = 0.0004633617354556

Epoch 1, batch 200, Loss = 0.0002704524958971888
Epoch 1, batch 201, Loss = 0.043007031083106995
Epoch 1, batch 202, Loss = 0.0002867126604542136
Epoch 1, batch 203, Loss = 0.0005224418709985912
Epoch 1, batch 204, Loss = 0.03255872800946236
Epoch 1, batch 205, Loss = 0.0006820106646046042
Epoch 1, batch 206, Loss = 0.00031765937455929816
Epoch 1, batch 207, Loss = 0.0004994773771613836
Epoch 1, batch 208, Loss = 0.0009714221814647317
Epoch 1, batch 209, Loss = 0.0018985748756676912
Epoch 1, batch 210, Loss = 0.0010189437307417393
Epoch 1, batch 211, Loss = 0.0006257438799366355
Epoch 1, batch 212, Loss = 0.00033204079954884946
Epoch 1, batch 213, Loss = 0.0011419105576351285
Epoch 1, batch 214, Loss = 0.00032423020456917584
Epoch 1, batch 215, Loss = 0.0003874969552271068
Epoch 1, batch 216, Loss = 0.0011980247218161821
Epoch 1, batch 217, Loss = 0.0012537384172901511
Epoch 1, batch 218, Loss = 0.0008062267443165183
Epoch 1, batch 219, Loss = 0.004053354263305664
Epoch 1, batch 220, L

...

Epoch 10, batch 382, Loss = 0.0001534843468107283
Epoch 10, batch 383, Loss = 9.552955452818424e-05
Epoch 10, batch 384, Loss = 8.56399565236643e-05
Epoch 10, batch 385, Loss = 7.570266461698338e-05
Epoch 10, batch 386, Loss = 0.00012004852032987401
Epoch 10, batch 387, Loss = 0.00014965057198423892
Epoch 10, batch 388, Loss = 0.00010740280413301662
Epoch 10, batch 389, Loss = 0.00016062735812738538
Epoch 10, batch 390, Loss = 0.00019158363284077495
Epoch 10, batch 391, Loss = 0.00016025542572606355
Epoch 10, batch 392, Loss = 0.0001163578053819947
Epoch 10, batch 393, Loss = 0.00010446548549225554
Epoch 10, batch 394, Loss = 0.00012273788161110133
Epoch 10, batch 395, Loss = 0.00010038376058219001
Epoch 10, batch 396, Loss = 9.32693510549143e-05
Epoch 10, batch 397, Loss = 0.00011667251237668097
Epoch 10, batch 398, Loss = 0.0002001857792492956
Epoch 10, batch 399, Loss = 0.00015401840209960938
Epoch 10, batch 400, Loss = 0.00011720657494151965
Epoch 10, batch 401, Loss = 0.0001604652

Epoch 11, batch 66, Loss = 0.00020786286040674895
Epoch 11, batch 67, Loss = 8.239746239269152e-05
Epoch 11, batch 68, Loss = 0.00014579773414880037
Epoch 11, batch 69, Loss = 0.00013668061001226306
Epoch 11, batch 70, Loss = 0.00019618988153524697
Epoch 11, batch 71, Loss = 6.761551048839465e-05
Epoch 11, batch 72, Loss = 0.00012653350131586194
Epoch 11, batch 73, Loss = 0.0001292800880037248
Epoch 11, batch 74, Loss = 6.62612947053276e-05
Epoch 11, batch 75, Loss = 0.00013977050548419356
Epoch 11, batch 76, Loss = 0.00011182785237906501
Epoch 11, batch 77, Loss = 0.00011131286737509072
Epoch 11, batch 78, Loss = 6.599425978492945e-05
Epoch 11, batch 79, Loss = 0.00017131805361714214
Epoch 11, batch 80, Loss = 0.0001224517764057964
Epoch 11, batch 81, Loss = 0.0001145458227256313
Epoch 11, batch 82, Loss = 0.00014547347382176667
Epoch 11, batch 83, Loss = 0.00011512756464071572
Epoch 11, batch 84, Loss = 0.00023656844859942794
Epoch 11, batch 85, Loss = 0.0001632213534321636
Epoch 11,

Epoch 11, batch 229, Loss = 0.0002284622169099748
Epoch 11, batch 230, Loss = 0.00019188881560694426
Epoch 11, batch 231, Loss = 0.00013607979053631425
Epoch 11, batch 232, Loss = 0.00023975371732376516
Epoch 11, batch 233, Loss = 0.0001650428748689592
Epoch 11, batch 234, Loss = 0.00011878967052325606
Epoch 11, batch 235, Loss = 0.00020599365234375
Epoch 11, batch 236, Loss = 0.00010568618745310232
Epoch 11, batch 237, Loss = 8.850097947288305e-05
Epoch 11, batch 238, Loss = 8.110046474030241e-05
Epoch 11, batch 239, Loss = 9.110450628213584e-05
Epoch 11, batch 240, Loss = 0.0001607036538189277
Epoch 11, batch 241, Loss = 0.00012509345833677799
Epoch 11, batch 242, Loss = 7.085800461936742e-05
Epoch 11, batch 243, Loss = 0.00010049819684354588
Epoch 11, batch 244, Loss = 0.00018894195090979338
Epoch 11, batch 245, Loss = 0.00015672683366574347
Epoch 11, batch 246, Loss = 0.00012424468877725303
Epoch 11, batch 247, Loss = 9.088516526389867e-05
Epoch 11, batch 248, Loss = 0.000144109726

Epoch 11, batch 392, Loss = 0.0001936435728566721
Epoch 11, batch 393, Loss = 0.0002436542563373223
Epoch 11, batch 394, Loss = 0.00013647080049850047
Epoch 11, batch 395, Loss = 0.0002542877336964011
Epoch 11, batch 396, Loss = 0.00010597229265840724
Epoch 11, batch 397, Loss = 0.00012300490925554186
Epoch 11, batch 398, Loss = 0.00024047851911745965
Epoch 11, batch 399, Loss = 8.230209641624242e-05
Epoch 11, batch 400, Loss = 7.640838884981349e-05
Epoch 11, batch 401, Loss = 0.0002087020839098841
Epoch 11, batch 402, Loss = 0.00011417389032430947
Epoch 11, batch 403, Loss = 7.480621570721269e-05
Epoch 11, batch 404, Loss = 0.00011157989501953125
Epoch 11, batch 405, Loss = 0.00018468857160769403
Epoch 11, batch 406, Loss = 9.04464686755091e-05
Epoch 11, batch 407, Loss = 0.00016840934404172003
Epoch 11, batch 408, Loss = 0.0001090240475605242
Epoch 11, batch 409, Loss = 9.399413829669356e-05
Epoch 11, batch 410, Loss = 0.00010209083848167211
Epoch 11, batch 411, Loss = 0.000108985899

Epoch 12, batch 77, Loss = 7.899284537415951e-05
Epoch 12, batch 78, Loss = 9.198188490699977e-05
Epoch 12, batch 79, Loss = 6.21414219494909e-05
Epoch 12, batch 80, Loss = 0.00011940002150367945
Epoch 12, batch 81, Loss = 0.00012589454127009958
Epoch 12, batch 82, Loss = 0.00019527434778865427
Epoch 12, batch 83, Loss = 0.00010179519449593499
Epoch 12, batch 84, Loss = 7.41958647267893e-05
Epoch 12, batch 85, Loss = 0.00013143540127202868
Epoch 12, batch 86, Loss = 0.00010144233965547755
Epoch 12, batch 87, Loss = 0.00011592864757403731
Epoch 12, batch 88, Loss = 0.0001315021509071812
Epoch 12, batch 89, Loss = 0.00011211395030841231
Epoch 12, batch 90, Loss = 0.0001617431698832661
Epoch 12, batch 91, Loss = 9.818076796364039e-05
Epoch 12, batch 92, Loss = 5.7449342421023175e-05
Epoch 12, batch 93, Loss = 0.00010315894905943424
Epoch 12, batch 94, Loss = 0.00021036148245912045
Epoch 12, batch 95, Loss = 8.903503476176411e-05
Epoch 12, batch 96, Loss = 0.00026859284844249487
Epoch 12, 

In [ ]:
71